In [1]:
from pdf2image import convert_from_path

In [125]:
pages = convert_from_path(r'docs\patient_details\pd_2.pdf',poppler_path=r'C:\poppler-24.02.0\Library\bin') # poppler used to change pdf 2 image

In [126]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1500x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1992x2000>]

In [127]:
pages[0].show()

In [128]:
import pytesseract

In [129]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe' #image to text conversion this tessarcat is engine
text= pytesseract.image_to_string(pages[0], lang='eng')
print(text)

a

Patient Medical Record

Patient Information
Jerry Lucas

(279) 920-8204
4218 Wheeler Ridge Dr

Buffalo, New York, 14201
United States

In Case of Emergency

Birth Date
May 2 1998

Weight:
57

Height:
170

Joe Lucas

Home phone

General Medical History

4218 Wheeler Ridge Dr
Buffalo, New York, 14201
United States

Work phone

Chicken Pox (Varicelia):

IMMUNE

Have you had the Hepatitis B vaccination?

Measles:

NOT IMMUNE

any Medical Problems (asthma, seizures, headaches):




In [130]:
import numpy as np
import cv2

def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5,fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image= cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        61,
        11
    )
    
    return processed_image

In [131]:
img = preprocess_image(pages[0])
from PIL import Image
Image.fromarray(img).show()

In [132]:

text= pytesseract.image_to_string(img, lang='eng')
print(text)

Patient Medical Record

Patient Information Birth Date

Jerry Lucas May 2 1998

(279) 920-8204 Weight:

4218 Wheeler Ridge Dr 57

Buffalo, New York, 14201 Height:

United States gnt
170

In Case of Emergency

eee

Joe Lucas . 4218 Wheeler Ridge Dr
Buffalo, New York, 14201
Home phone United States
Work phone

General Medical History

Chicken Pox (Varicelia): Measles: ..

IMMUNE NOT IMMUNE

Have you had the Hepatitis B vaccination?

‘Yes

| List any Medical Problems (asthma, seizures, headaches):
N/A

7?
v

17/12/2020




In [106]:
import re

In [107]:
pattern = 'Patient.Information(.*?)\(\d{3}\)'

matches = re.findall(pattern, text,flags=re.DOTALL)
matches


<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\DELL\AppData\Local\Temp\ipykernel_1724\727179948.py:1: SyntaxWarning: invalid escape sequence '\('
  pattern = 'Patient.Information(.*?)\(\d{3}\)'


[' Birth Date\nKathy Crawford May 6 1972\n']

In [108]:
matches[0].strip()

'Birth Date\nKathy Crawford May 6 1972'

In [109]:
match = matches[0].replace("Birth Date","").strip()
match

'Kathy Crawford May 6 1972'

In [110]:
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
date_matches = re.findall(pattern,match)
date= date_matches[0][0]

date

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\DELL\AppData\Local\Temp\ipykernel_1724\3616450927.py:1: SyntaxWarning: invalid escape sequence '\d'
  pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'


'May 6 1972'

In [111]:
match.replace(date, '').strip()

'Kathy Crawford'

In [112]:
def remove_noise_from_name(name):
    name=name.replace('Birth Date','').strip()
    date_pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern,match)

    if date_matches:
        date = date_matches[0][0]
        name=name.replace(date,'').strip()
    return name

    
   

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\DELL\AppData\Local\Temp\ipykernel_1724\3078535572.py:3: SyntaxWarning: invalid escape sequence '\d'
  date_pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'


In [113]:
name= ' Birth Date\nKathy Crawford May 6 1972\n'
name = remove_noise_from_name(name)
name

'Kathy Crawford'

In [133]:
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'
matches = re.findall(pattern, text,flags=re.DOTALL)
matches



<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\DELL\AppData\Local\Temp\ipykernel_1724\1428127916.py:1: SyntaxWarning: invalid escape sequence '\('
  pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'


[(' Birth Date\n\nJerry Lucas May 2 1998\n\n', '(279) 920-8204')]

In [115]:
matches[0][1]

'(737) 988-0851'

In [116]:
pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'
matches = re.findall(pattern, text,flags=re.DOTALL)
matches


<>:1: SyntaxWarning: invalid escape sequence '\?'
<>:1: SyntaxWarning: invalid escape sequence '\?'
C:\Users\DELL\AppData\Local\Temp\ipykernel_1724\1036899923.py:1: SyntaxWarning: invalid escape sequence '\?'
  pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'


['No']

In [123]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\}:(.*)'
matches = re.findall(pattern, text,flags=re.DOTALL)
matches


<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\DELL\AppData\Local\Temp\ipykernel_1724\2047859999.py:1: SyntaxWarning: invalid escape sequence '\('
  pattern = 'List any Medical Problems \(asthma, seizures, headaches\}:(.*)'


['\n\nMigraine\n\nbe\n\nCO\nnat\naa\noo\n']

In [124]:
matches[0].strip()

'Migraine\n\nbe\n\nCO\nnat\naa\noo'